In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hello, this is my final project for the course Intro to AI. I will try to use AI in order to predict the evolution of Bitcoin.

### Set Up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn 

In [ ]:
#usefull when cells take time
def progressBar(current, total, barLength = 20):
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))

    print('Progress: [%s%s] %d %%' % (arrow, spaces, percent), end='\r')

### My simple database

In [ ]:
bitcoin = pd.read_csv('Bitcoin_data.csv')
bitcoin
#evolution of bitcoin per hours, with 28587 hours

In [ ]:
bitcoin = bitcoin.reindex(index=bitcoin.index[::-1])
bitcoin.reset_index(inplace=True, drop=True)
bitcoin

In [ ]:
bitcoin.info()

Result_BTC is the pourcentage of the increased value of bitcoin for the next hour. Classification is 1 if the value increased the next hour, 0 if it stagnates and -1 if it descreases.  

In [ ]:
bitcoin.describe()

In [ ]:
train_set = bitcoin.iloc[:28000, :]
test_set = bitcoin.iloc[28001:, :]

In [ ]:
corr_matrix = train_set.corr()
corr_matrix["Classification"].sort_values(ascending=False)

In [ ]:
corr_matrix["Result"].sort_values(ascending=False)

Correlations are not very good, but later I will find interesting results nonetheless.

## Preparation for machine learning

### Trying to predict the exact result

In [ ]:
X_train = train_set[["Open","High","Low","Close"]].values
X_test = test_set[["Open","High","Low","Close"]].values
X_train
#after several try, I decided to keep only these features

In [ ]:
y_train = train_set["Result"].values
y_test = test_set["Result"].values

y_train

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

bitcoin_train_predictions = lin_reg.predict(X_train)
bitcoin_train_predictions

from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_train, bitcoin_train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# CROSS VALIDATION
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
lin_scores = cross_val_score(lin_reg, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

The error is to large

### Decision tree regressor

In [ ]:
x, y = bitcoin[["Open","High","Low","Close"]], bitcoin['Result']
x_train = X_train
x_valid = X_test
y_valid = y_test

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(x_train,y_train)
model.score(x_train,y_train), model.score(x_valid,y_valid)

The model seems to overfit the data. Let's try to regularize it.

In [ ]:
y_hat = model.predict(x_valid)
y_m = y_valid.mean()

ss_res = ((y_valid - y_hat)**2).sum()
ss_tot = ((y_valid - y_m)**2).sum()
r_squared = 1 - ss_res/ss_tot
r_squared

In [ ]:
score_train, score_valid = [],[]
msl = [i for i in range(1,15)]
for m in msl:
    model = DecisionTreeRegressor(min_samples_leaf=m)
    model.fit(x_train,y_train)
    print(m, model.score(x_train,y_train), model.score(x_valid,y_valid))
    score_train.append(model.score(x_train,y_train))
    score_valid.append(model.score(x_valid,y_valid))

In [ ]:
plt.plot(msl, score_train)
plt.plot(msl, score_valid)

In [ ]:
# CROSS VALIDATION

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)


display_scores(tree_rmse_scores)

The model perform poorly, even regularized.

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, y_train)
bitcoin_train_predictions = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, bitcoin_train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
#CROSS VALIDATION

#from sklearn.model_selection import cross_val_score

#forest_scores = cross_val_score(forest_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
#forest_rmse_scores = np.sqrt(-forest_scores)
#display_scores(forest_rmse_scores)

Uncomment to execute this cross validation, but it takes a lot of time and the error is still to important

## In this part we will use classification models in order to try to predict if the bitcoin will increase (True) or not (False)

In [ ]:
bitcoin['Increase'] = np.where(bitcoin['Result']>0.0, True, False)
bitcoin.head()

In [ ]:
len(bitcoin)

In [ ]:
train_set = bitcoin.iloc[24586:24986]
test_set_1 = bitcoin.iloc[24987:25706]
test_set_2 = bitcoin.iloc[25707:26426]
test_set_3 = bitcoin.iloc[26427:27146]
test_set_4 = bitcoin.iloc[27147:27866]
test_set_5 = bitcoin.iloc[27867:, :]
#5 last months for testing

In [ ]:
train_set

In [ ]:
X_train = train_set[["Open","High","Low","Close"]].values
X_train

In [ ]:
y_train = train_set["Increase"].values
y_train

In [ ]:
X_test_1 = test_set_1[["Open","High","Low","Close"]].values
X_test_2 = test_set_2[["Open","High","Low","Close"]].values
X_test_3 = test_set_3[["Open","High","Low","Close"]].values
X_test_4 = test_set_4[["Open","High","Low","Close"]].values
X_test_5 = test_set_5[["Open","High","Low","Close"]].values
X_test_1

In [ ]:
y_test_1 = test_set_1["Increase"].values
y_test_result_1 = test_set_1["Result"].values
y_test_2 = test_set_2["Increase"].values
y_test_result_2 = test_set_2["Result"].values
y_test_3 = test_set_3["Increase"].values
y_test_result_3 = test_set_3["Result"].values
y_test_4 = test_set_4["Increase"].values
y_test_result_4 = test_set_4["Result"].values
y_test_5 = test_set_5["Increase"].values
y_test_result_5 = test_set_5["Result"].values

In [ ]:
i=0
while i < len(y_train):
    print(y_test_result_1[i], y_test_1[i])
    i+=1

### Simulation building

In this part, we will build a simulation method, where we invest every hour based on our predictions. We can invest on the increase or the decrease of the stock, and we fix a stop loss below which we sell, and a take profit when we sell also.

In [ ]:
def simulation(X_test, y_test_result, starting_wallet, stop_loss, take_profit, clf):
    i=0
    tendance = 0
    y_result = y_test_result
    total = starting_wallet
    while i < len(y_result):
        
        #si l'ia nous dit de miser la hausse, qu'on mise la hausse et que le cours augmente
        #if the model tells us that the stock will increase and it's true
        if clf.predict(X_test)[i] == True and y_result[i] > 0:
            tendance = 1
            if y_result[i] > (take_profit):
                total =  (tendance * ((1 + (take_profit/100)) * total))
                commission = 0.0004 * total
                total = total - commission
            else:
                total =  (tendance * ((1 + (y_result[i]/100)) * total))
                commission = 0.0004 * total
                total = total - commission

        #si l'ia nous dit de miser la hausse, qu'on mise la hausse et que le cours descend
        #if the model tells us that the stock will increase and it's not true
        elif clf.predict(X_test)[i] == True and y_result[i] < 0:
            tendance = -1
            if y_result[i] < (stop_loss):
                total =  ((1 - (-stop_loss/100)) * total)
                commission = 0.0004 * total
                total = total - commission
            else:
                total =  ((1 - (-y_result[i]/100)) * total)
                commission = 0.0004 * total
                total = total - commission

        #si l'ia nous dit de miser la baisse, qu'on mise le baisse et que le cours augmente
        #if the model tells us that the stock will decrease and it's not true
        elif clf.predict(X_test)[i] == False and y_result[i] > 0:
            tendance = -1
            if (-y_result[i]) < stop_loss:
                total =  ((1 - (-stop_loss/100)) * total)
                commission = 0.0004 * total
                total = total - commission
            else:
                total =  ((1 - (y_result[i]/100)) * total)
                commission = 0.0004 * total
                total = total - commission

        #si l'ia nous dit de miser la baisse, qu'on mise la baisse et que le cours descend
        #if the model tells us that the stock will decrease and it's true, we decide to "short"
        elif clf.predict(X_test)[i] == False and y_result[i] < 0:
            tendance = 1
            if y_result[i] < (-take_profit):
                total =  (tendance * (1 + (take_profit/100)) * total)
                commission = 0.0004 * total
                total = total - commission
            else:
                total =  (tendance * ((1 + (-y_result[i]/100))) * total)
                commission = 0.0004 * total
                total = total - commission

        i+=1
    print('Wallet at the begining of the test simulation : ',starting_wallet)
    print('Wallet at the end of the test simulation : ',round(total, 2))
    print('---------------------------')
    return round(total, 2)

In [ ]:
def detailed_simulation(X_test, y_test_result, starting_wallet, stop_loss, take_profit, clf):
    i=0
    tendance = 0
    y_result = y_test_result
    total = starting_wallet
    print('Stop loss : ',stop_loss)
    print('Take profit : ',take_profit)
    print('---- BEGINNING SIMULATION ----')
    while i < len(y_result):
        print('Predicted : ', clf.predict(X_test)[i])
        print('Reality : ', y_result[i])

        #si l'ia nous dit de miser la hausse, qu'on mise la hausse et que le cours augmente
        #if the model tells us that the stock will increase and it's true
        if clf.predict(X_test)[i] == True and y_result[i] > 0:
            tendance = 1
            if y_result[i] > (take_profit):
                print('Take profit : ', take_profit)
                total =  (tendance * ((1 + (take_profit/100)) * total))
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)
            else:
                total =  (tendance * ((1 + (y_result[i]/100)) * total))
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)

        #si l'ia nous dit de miser la hausse, qu'on mise la hausse et que le cours descend
        #if the model tells us that the stock will increase and it's not true
        elif clf.predict(X_test)[i] == True and y_result[i] < 0:
            tendance = -1
            if y_result[i] < (stop_loss):
                print('Stop loss : ', stop_loss)
                total =  ((1 - (-stop_loss/100)) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)
            else:
                total =  ((1 - (-y_result[i]/100)) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)

        #si l'ia nous dit de miser la baisse, qu'on mise le baisse et que le cours augmente
        #if the model tells us that the stock will decrease and it's not true
        elif clf.predict(X_test)[i] == False and y_result[i] > 0:
            tendance = -1
            if (-y_result[i]) < stop_loss:
                print('Stop loss : ', stop_loss)
                total =  ((1 - (-stop_loss/100)) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)
            else:
                total =  ((1 - (y_result[i]/100)) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)

        #si l'ia nous dit de miser la baisse, qu'on mise la baisse et que le cours descend
        #if the model tells us that the stock will decrease and it's true, we decide to "short"
        elif clf.predict(X_test)[i] == False and y_result[i] < 0:
            tendance = 1
            if y_result[i] < (-take_profit):
                print('Take profit : ', take_profit)
                total =  (tendance * (1 + (take_profit/100)) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)
            else:
                total =  (tendance * ((1 + (-y_result[i]/100))) * total)
                print('Wallet before commission : ', total)
                commission = 0.0004 * total
                print('Commission : ', commission)
                total = total - commission
                print('Wallet after commission : ', total)

        print('---------------------------')
        i+=1
        #progressBar(i, len(y_result))
    print("")
    print('END OF SIMULATION')
    print('Wallet at the begining of the test simulation : ',starting_wallet)
    print('Wallet at the end of the test simulation : ',round(total, 2))
    print('---------------------------')
    return round(total, 2)

### Stochastic Gradient Descent (SGD) classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier().fit(X_train, y_train)
i = 0
good_predictions = 0
total_predictions = len(X_train)
while i < total_predictions:
    #print(sgd_clf.predict([X_train[i]]), y_train[i])
    if (sgd_clf.predict([X_train[i]]) == True and y_train[i] == 1):
        good_predictions += 1
    elif (sgd_clf.predict([X_train[i]]) == False and y_train[i] != 1):
        good_predictions += 1
    i += 1
accuracy = good_predictions * 100 / total_predictions
print('number of good predictions = ', good_predictions)
print ('total predictions = ', total_predictions)
print ('accuracy = ', accuracy)

Let's see how the model perform on the 5 mounths

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, sgd_clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, sgd_clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, sgd_clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, sgd_clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, sgd_clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, sgd_clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, sgd_clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, sgd_clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, sgd_clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, sgd_clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, sgd_clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, sgd_clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, sgd_clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, sgd_clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, sgd_clf.predict(X_test_5))))

Let's focus on the fourth mounth

In [ ]:
index = 0
count_tp = 0
count_fp = 0 
count_fn = 0 
count_tn = 0
count_negative = 0
count_positive = 0 
for i in sgd_clf.predict(X_test_4):
    if i == True:
        if y_test_4[index] == True:
            count_tp+=1
        if y_test_4[index] == False:
            count_fp+=1
    if i == False:
        if y_test_4[index] == True:
            count_fn+=1
        if y_test_4[index] == False:
            count_tn+=1
    index+=1

for i in y_test_4:
    if i == True:
        count_positive+=1
    else:
        count_negative+=1
        
print('True positives :', count_tp)
print('False positives :', count_fp)
print('True negatives :', count_tn)
print('False negatives :', count_fn)


If we run a detailed simulation on the second mounth, we can see what is going on step by step if we try this model in real life. We will use a stop loss of -1% and a take profit of 1.2%, with 10000USD.

In [ ]:
detailed_simulation(X_test_2, y_test_result_2, 10000, -1, 1.2, sgd_clf)

We see that we loose money. Now let's try on the 5 last mounth : 

In [ ]:
print('Mounth 1')
total_1 = simulation(X_test_1, y_test_result_1, 10000, -1, 1.2, sgd_clf)
print('Mounth 2')
total_2 = simulation(X_test_2, y_test_result_2, total_1, -1, 1.2, sgd_clf)
print('Mounth 3')
total_3 = simulation(X_test_3, y_test_result_3, total_2, -1, 1.2, sgd_clf)
print('Mounth 4')
total_4 = simulation(X_test_4, y_test_result_4, total_3, -1, 1.2, sgd_clf)
print('Mounth 5')
total_5 = simulation(X_test_5, y_test_result_5, total_4, -1, 1.2, sgd_clf)

In order to improve our simulation we should calculate also when during the hour, the stock reach the stop loss and end at a higher value. We will ignore this problem in this final project, for time. Nevertheless, it's very rare that the stock reach -1% and end at a higher value.

## KNN classifier


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_class = KNeighborsClassifier(n_neighbors=4, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)
knn_class.fit(X_train, y_train)

print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, knn_class.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, knn_class.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, knn_class.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, knn_class.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, knn_class.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, knn_class.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, knn_class.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, knn_class.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, knn_class.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, knn_class.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, knn_class.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, knn_class.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, knn_class.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, knn_class.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, knn_class.predict(X_test_5))))

In [ ]:
print('Mounth 1')
total_1 = simulation(X_test_1, y_test_result_1, 10000, -1, 1.2, knn_class)
print('Mounth 2')
total_2 = simulation(X_test_2, y_test_result_2, total_1, -1, 1.2, knn_class)
print('Mounth 3')
total_3 = simulation(X_test_3, y_test_result_3, total_2, -1, 1.2, knn_class)
print('Mounth 4')
total_4 = simulation(X_test_4, y_test_result_4, total_3, -1, 1.2, knn_class)
print('Mounth 5')
total_5 = simulation(X_test_5, y_test_result_5, total_4, -1, 1.2, knn_class)

## Random Forest Classifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, clf.predict(X_test_5))))

In [ ]:
print('Mounth 1')
total_1 = simulation(X_test_1, y_test_result_1, 10000, -1, 1.2, clf)
print('Mounth 2')
total_2 = simulation(X_test_2, y_test_result_2, total_1, -1, 1.2, clf)
print('Mounth 3')
total_3 = simulation(X_test_3, y_test_result_3, total_2, -1, 1.2, clf)
print('Mounth 4')
total_4 = simulation(X_test_4, y_test_result_4, total_3, -1, 1.2, clf)
print('Mounth 5')
total_5 = simulation(X_test_5, y_test_result_5, total_4, -1, 1.2, clf)

### Fine tuning

Based on previous simulations, the best model is the SGD and the accuracy of the model have the biggest impact on the simulation. So we'll try to fine tune based on the documentation : https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py.

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
clf = SGDClassifier(loss='hinge', penalty='elasticnet', fit_intercept=True)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)


In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(alpha=0.001, average=False, l1_ratio=0.2222222222222222).fit(X_train, y_train)

In [ ]:
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, sgd_clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, sgd_clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, sgd_clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, sgd_clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, sgd_clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, sgd_clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, sgd_clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, sgd_clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, sgd_clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, sgd_clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, sgd_clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, sgd_clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, sgd_clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, sgd_clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, sgd_clf.predict(X_test_5))))

In [ ]:
print('Mounth 1')
total_1 = simulation(X_test_1, y_test_result_1, 10000, -0.5, 1.2, sgd_clf)
print('Mounth 2')
total_2 = simulation(X_test_2, y_test_result_2, total_1, -0.5, 1.2, sgd_clf)
print('Mounth 3')
total_3 = simulation(X_test_3, y_test_result_3, total_2, -0.5, 1.2, sgd_clf)
print('Mounth 4')
total_4 = simulation(X_test_4, y_test_result_4, total_3, -0.5, 1.2, sgd_clf)
print('Mounth 5')
total_5 = simulation(X_test_5, y_test_result_5, total_4, -0.5, 1.2, sgd_clf)

We can see that we improve a lot the performance of our model with the fine tuning. But as we said earlier, the simulation is not perfect, and the performance seems very random.

### Features engineering

I developed a new database with trading metrics and other crypto stocks

In [ ]:
bitcoin = pd.read_csv('bitcoin_fe.csv')
bitcoin

In [ ]:
bitcoin.info()

We have to clean the database (remove the rows '-' and NaN)

In [ ]:
bitcoin_to_remove = (bitcoin.loc[bitcoin['VolumeBTC'] == '-'])
bitcoin = bitcoin[~bitcoin.index.isin(bitcoin_to_remove.index)]
bitcoin_to_remove = (bitcoin.loc[bitcoin['VolumeUSDT_BTC'] == '-'])
bitcoin = bitcoin[~bitcoin.index.isin(bitcoin_to_remove.index)]

In [ ]:
len(bitcoin)

In [ ]:
bitcoin = bitcoin.apply (pd.to_numeric, errors='coerce')
bitcoin = bitcoin.drop(['Date'], axis = 1)
bitcoin = bitcoin.dropna()

In [ ]:
bitcoin.isnull().sum().sum()

In [ ]:
len(bitcoin)

In [ ]:
bitcoin.head()

Now that our new database is clean, let's prepare our data for classifications models.

In [ ]:
bitcoin['Increase'] = np.where(bitcoin['Result_BTC']>0.0, True, False)
bitcoin = bitcoin.drop(['Result_BTC'], axis = 1)
bitcoin.head()

In [ ]:
bitcoin.info()

In [ ]:
len(bitcoin)

In [ ]:
size = (len(bitcoin)-720)

test_set_5 = bitcoin.iloc[(size): len(bitcoin)]
size = size - 721
test_set_4 = bitcoin.iloc[(size+1):(size + 721)]
size = size - 721
test_set_3 = bitcoin.iloc[(size+1):(size + 721)]
size = size - 721
test_set_2 = bitcoin.iloc[(size+1):(size + 721)]
size = size - 721
test_set_1 = bitcoin.iloc[(size+1):(size + 721)]
size = size - 721
train_set = bitcoin.iloc[0:(size + 722)]
#5 last months for testing

In [ ]:
train_set

In [ ]:
test_set_5.info()

As you can see, I added a lot of features such as EMA (trading metric) and stocks of Etherium (another cryptocurrency)

In [ ]:
X_train = train_set.iloc[:,1:-2].values
X_test_1 = test_set_1.iloc[:,1:-2].values
X_test_2 = test_set_2.iloc[:,1:-2].values
X_test_3 = test_set_3.iloc[:,1:-2].values
X_test_4 = test_set_4.iloc[:,1:-2].values
X_test_5 = test_set_5.iloc[:,1:-2].values
y_train = train_set["Increase"].values
y_test_1 = test_set_1["Increase"].values
y_test_2 = test_set_2["Increase"].values
y_test_3 = test_set_3["Increase"].values
y_test_4 = test_set_4["Increase"].values
y_test_5 = test_set_5["Increase"].values

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, sgd_clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, sgd_clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, sgd_clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, sgd_clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, sgd_clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, sgd_clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, sgd_clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, sgd_clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, sgd_clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, sgd_clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, sgd_clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, sgd_clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, sgd_clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, sgd_clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, sgd_clf.predict(X_test_5))))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_class = KNeighborsClassifier(n_neighbors=4, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)
knn_class.fit(X_train, y_train)

print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, knn_class.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, knn_class.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, knn_class.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, knn_class.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, knn_class.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, knn_class.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, knn_class.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, knn_class.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, knn_class.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, knn_class.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, knn_class.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, knn_class.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, knn_class.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, knn_class.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, knn_class.predict(X_test_5))))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, clf.predict(X_test_5))))

Even with the fine tuning, the models seem to do randomly.

### Fine tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
clf = SGDClassifier(loss='hinge', penalty='elasticnet', fit_intercept=True)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(alpha=0.01, average=True, l1_ratio=0.5555555555555556).fit(X_train, y_train)

In [ ]:
print('Mounth 1:')
print('Accuracy: ', (100 * accuracy_score(y_test_1, sgd_clf.predict(X_test_1))))
print('Recall: ', (100 * recall_score(y_test_1, sgd_clf.predict(X_test_1))))
print('F1: ', (100 * f1_score(y_test_1, sgd_clf.predict(X_test_1))))
print('-----------------')
print('Mounth 2:')
print('Accuracy: ', (100 * accuracy_score(y_test_2, sgd_clf.predict(X_test_2))))
print('Recall: ', (100 * recall_score(y_test_2, sgd_clf.predict(X_test_2))))
print('F1: ', (100 * f1_score(y_test_2, sgd_clf.predict(X_test_2))))
print('-----------------')
print('Mounth 3:')
print('Accuracy: ', (100 * accuracy_score(y_test_3, sgd_clf.predict(X_test_3))))
print('Recall: ', (100 * recall_score(y_test_3, sgd_clf.predict(X_test_3))))
print('F1: ', (100 * f1_score(y_test_3, sgd_clf.predict(X_test_3))))
print('-----------------')
print('Mounth 4:')
print('Accuracy: ', (100 * accuracy_score(y_test_4, sgd_clf.predict(X_test_4))))
print('Recall: ', (100 * recall_score(y_test_4, sgd_clf.predict(X_test_4))))
print('F1: ', (100 * f1_score(y_test_4, sgd_clf.predict(X_test_4))))
print('-----------------')
print('Mounth 5:')
print('Accuracy: ', (100 * accuracy_score(y_test_5, sgd_clf.predict(X_test_5))))
print('Recall: ', (100 * recall_score(y_test_5, sgd_clf.predict(X_test_5))))
print('F1: ', (100 * f1_score(y_test_5, sgd_clf.predict(X_test_5))))

The model is still doing poorly, even with all the features we added.

### To go further

I did research on the web, and I found an interesting video that presents the LSTM (neural network) model. I do not recommand to launch these cells (I tested on google Collab, it lasted 8 hours, in order to save my computer) and I found interesting results : 

In [ ]:
import math
import numpy as np
import pandas as pd
#import pandas_datareader as pdd
from sklearn.preprocessing import MinMaxScaler
#from keras.models import Sequential
#from keras.layers.core import Dense, Activation, Dropout
#from keras.layers.recurrent import LSTM
import matplotlib.pyplot as plt

In [ ]:
import io
bitcoin_m = pd.read_csv('Bitcoin_1m_data.csv')
bitcoin_m.head

In [ ]:
bitcoin_m.info()

In [ ]:
bitcoin_m = bitcoin_m.iloc[400000:16725692 :]
data = bitcoin_m.filter(['Close'])

In [ ]:
dataset=data.dropna()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In order for our model to work, we will create several windows of 60 closing prices in order to predict the number 61. 

In [ ]:
train_data = scaled_data[0:training_data_len,:] 
x_train = []
y_train = []

for i in range(60, len(train_data)-60):
    x_train.append(train_data[i-60:i, 0])
    #y_train.append(train_data[i+60, 0])
    if (train_data[i+60, 0])>0:
      y_train.append(True)
    else:
      y_train.append(False)
    if i<= 60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

The lstm need a 3D x_train in order to work.

In [ ]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
test_data = scaled_data[training_data_len - 60: , :]

#create data sests x_tesst and y 
x_test = []
y_test = dataset[training_data_len:, :]
for i in range (60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

Last rmse obtained = 8.27602340010901

Even though the rmse change everytime we launch the model, 8.28 error in the estimation of Close is very low, and it's maybe worth to deep into it. 

### Conclusion

In this final project, I realize that trying to predict the stock of Bitcoin is very hard, and maybe beyond the scope of this course. Nevertheless, I tried to apply everything we learned and to interpret it, and why it's not working correctly. It's safe to assume that deep learning is a good lead to investigate for this prediction problem. Thank you.